In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from pathlib import Path
from scipy import special

eval_data = Path.home()/'group'/'project'/'scripting'/'output'/\
'220902_eval_cnn1d'/'model1'/'model1.npz'

eval_data = np.load(eval_data)

print([*eval_data.keys()])
print(eval_data['labels'])
print(special.softmax(eval_data['output'], axis=-1))

# separate: target signal inds, background signal inds, noise inds
n_target = 24902
n_back = 12451
n_noise = 12451
n_all = n_target + n_back + n_noise

target_mask = np.zeros(n_all, dtype=bool)
target_mask[0:n_target]=1

back_mask = np.zeros(n_all, dtype=bool)
back_mask[n_target:n_target+n_back]=1

noise_mask = np.zeros(n_all, dtype=bool)
noise_mask[n_target+n_back:] = 1

target_mask = np.tile(target_mask, 150)
back_mask = np.tile(back_mask, 150)
noise_mask = np.tile(noise_mask, 150)


# pitch angle selection #
# get the pitch angles we want
pitch_mask = np.concatenate(
    (
        np.logical_and(eval_data['train_pitch']<=86.65, eval_data['train_pitch']>=86.5),
        np.zeros(eval_data['train_pitch'].size, dtype=bool)
    )
)
pitch_mask = np.tile(pitch_mask, 150)

# get the background and noise
#noise_mask = np.logical_not(eval_data['labels'])
#print(noise_mask.sum(), noise_mask.sum()//150)

# we want both noise and the select pitches
#mask = pitch_mask + noise_mask

#print(mask.sum())



In [ ]:
sns.set_theme(context='talk', style='whitegrid', )
fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot(1,1,1)

x = ax.hist(special.softmax(eval_data['output'][eval_data['labels']==1, :], axis=-1)[:, 1], bins=np.linspace(0, 1, 101)) # label is 1 output 1
x = ax.hist(special.softmax(eval_data['output'][eval_data['labels']==0, :], axis=-1)[:, 1], bins=np.linspace(0, 1, 101)) # label is 0 output 1

In [ ]:
sns.set_theme(context='talk', style='whitegrid', )
fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot(1,1,1)

x = ax.hist(
    special.softmax(eval_data['output'][target_mask, :], axis=-1)[:, 1],
    bins=np.linspace(0, 1, 101),
    histtype='step',
    density=True
) # label is 1 output 1

x = ax.hist(
    special.softmax(eval_data['output'][pitch_mask, :], axis=-1)[:, 1],
    bins=np.linspace(0, 1, 101),
    histtype='step',
    density=True,
) # label is 1 output 1


x = ax.hist(
    special.softmax(eval_data['output'][noise_mask, :], axis=-1)[:, 1],
    bins=np.linspace(0, 1, 101),
    histtype='step',
    density=True,
) # label is 0 output 1
x = ax.hist(
    special.softmax(eval_data['output'][back_mask, :], axis=-1)[:, 1],
    bins=np.linspace(0, 1, 101),
    histtype='step',
    density=True
) # label is 1 output 1

ax.set_xlim(0.7, 0.9)
ax.set_ylim(0, 4)


In [ ]:

fpr, tpr, thresholds = metrics.roc_curve(
    eval_data['labels'],
    #eval_data['output'][:, 1], 
    special.softmax(eval_data['output'], axis=-1)[:, 1]
)

fpr_mask, tpr_mask, thresholds = metrics.roc_curve(
    eval_data['labels'][mask],
    #eval_data['output'][:, 1], 
    special.softmax(eval_data['output'], axis=-1)[mask, 1]
)


sns.set_theme(style='whitegrid', context='talk', font_scale=1.2 )
fig = plt.figure(figsize=(8, 5))

ax = fig.add_subplot(1,1,1)
ax.plot(fpr, tpr)
ax.plot(fpr_mask, tpr_mask)
ax.plot(np.linspace(0, 1, 1001), np.linspace(0, 1, 1001))
#plt.xscale('log')